In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import utils,preprocess,merge_base
import time
import datetime
import os,gc
file_path='D:/360WiFi/'
raw_data_path='D:/360WiFi/FDDC_financial_data_20180613/'
split_file_path='D:/360WiFi/files/financial_data_new/'
input_path='D:/360WiFi/files/input/'
mid_path='D:/360WiFi/files/mid_file/'
other_path='D:/360WiFi/files/other_file/'



def merge_3file():
    '''
    输入是合好的三大财务表，输出是比赛给的六表合并
    
    '''
    
    ###merge Market Data 数据

    table_base=pd.read_csv(mid_path+'table_base.csv')
    market_data=pd.read_excel(other_path+'[New] Market Data_20180613.xlsx')
    market_data['year']=market_data['END_DATE_'].dt.year
    market_data['month']=market_data['END_DATE_'].dt.month
    market_data.columns

    table_base['year']=table_base['END_DATE'].map(lambda x:'%s'%str(x)[0:4]).astype(int)
    table_base['month']=table_base['END_DATE'].map(lambda x:'%s'%str(x)[5:7]).astype(int)
    table_base

    merge1=pd.merge(table_base,market_data[[ 'TICKER_SYMBOL', 'CLOSE_PRICE', 'TURNOVER_VOL', 'TURNOVER_VALUE', 'MARKET_VALUE',\
                                            'year', 'month']],how='left',on=['TICKER_SYMBOL','year','month'])
    merge1


    ###merge Macro Data 数据
    macro_factor=pd.read_excel(other_path+'[New] Macro&Industry_20180613.xlsx',sheetname='指标信息',encoding='gbk')
    macro_factor_list=macro_factor.INDIC_ID.astype(str).values

    macro_data=pd.read_excel(other_path+'[New] Macro&Industry_20180613.xlsx',sheetname='数据信息',encoding='gbk')
    macro_data=macro_data[macro_data.PERIOD_DATE>='2009-01-01']
    macro_data.indic_id=macro_data.indic_id.astype(str)

    df_list=[]

    for factor in macro_factor_list:  
        #print(factor)
        t=macro_data[macro_data.indic_id==factor][['PERIOD_DATE','DATA_VALUE']]
        t.rename(columns={'DATA_VALUE':factor},inplace=True)
        df_list.append(t)
    M=df_list[0]
    for i in range(1,len(df_list)):
        M=pd.merge(M,df_list[i],how='outer',on=['PERIOD_DATE'])
    M.drop_duplicates(inplace=True)
    M.to_csv(other_path+'macro_data.csv',index=False)


    M['year']=M['PERIOD_DATE'].map(lambda x:'%s'%str(x)[0:4]).astype(int)
    M['month']=M['PERIOD_DATE'].map(lambda x:'%s'%str(x)[5:7]).astype(int)
    M
    M['PERIOD_DATE']=pd.to_datetime(M['PERIOD_DATE'],format='%Y-%m-%d') 
    M.sort_values(by=['PERIOD_DATE'],inplace=True)
    M.drop_duplicates(subset=['year','month'],inplace=True,keep='last')


    merge2=pd.merge(merge1,M[M.columns[1:]],how='left',on=['year','month'])
    merge2


    ###merge Company operation Data 数据
    company_operation=pd.read_excel(other_path+'[New] Company Operating_2018613.xlsx',sheetname='CN')
    company_operation.rename(columns={'交易代码':'TICKER_SYMBOL','报告期':'END_DATE','值':'VALUE','指标':'INDIC_NAME_CN'},inplace=True)
    company_operation.drop(company_operation.index[[0]],inplace=True)
    company_operation.TICKER_SYMBOL=company_operation.TICKER_SYMBOL.astype(str)
    company_operation['year']=company_operation['END_DATE'].dt.year.fillna(0).astype(int)
    company_operation['month']=company_operation['END_DATE'].dt.month.fillna(0).astype(int)
    company_operation=company_operation[company_operation.year>2008]
    company_operation=company_operation[['TICKER_SYMBOL','VALUE','INDIC_NAME_CN','year','month']]

    a=company_operation.groupby(['TICKER_SYMBOL','INDIC_NAME_CN']).size().reset_index()
    a

    ticker_INDIC=zip(a.TICKER_SYMBOL,a.INDIC_NAME_CN)
    merge_list=[]

    for (a,b) in ticker_INDIC:

        t=company_operation[(company_operation['TICKER_SYMBOL']==a)&(company_operation['INDIC_NAME_CN']==b)]
        t.rename(columns={'VALUE':a+'_'+b},inplace=True)
        merge_list.append(t)

    M=merge_list[0]
    for i in range(1,len(merge_list)):
        merge_list[i].drop(['INDIC_NAME_CN'],axis=1,inplace=True)
        M=pd.merge(M,merge_list[i],how='outer',on=['TICKER_SYMBOL','year','month'])

    M.drop(['INDIC_NAME_CN'],axis=1,inplace=True)
    M
    M.TICKER_SYMBOL=M.TICKER_SYMBOL.astype(int)

    merge3=pd.merge(merge2,M,how='left',on=['TICKER_SYMBOL','year','month'])
    merge3


    merge3.to_csv(mid_path+'table_6_file.csv',index=False)
    
    print('the shape of 6_sheet merge_file:',merge3.shape)
    
    return 